# import library

In [38]:
import pandas as pd
from sklearn.datasets import load_boston
import xgboost
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
import random
import os
import numpy as np
from sklearn import metrics 
#    abnormal_data = pd.read_csv("./abnormal_data/Y_%02d.csv"%i)

In [39]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [40]:
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score

In [43]:
# 범주 내 데이터들만 가지고 딥러닝 학습 시키기
# 참고로 예측하고자 하는 Y의 컬럼별로 학습시 요구되는 X컬럼들의 양이 다르다.(범주 밖은 제거했기 때문)
    # print("Y_%02d"%i,"를 설명하는 중요한 X 변수")
    
    # xgboost.plot_importance(xgb_model)
Y_pred = pd.read_csv("./sample_submission.csv")['ID']
test_x = pd.read_csv("./test.csv").drop(columns=['ID'])

for i in range(1,15):
    normal_data = pd.read_csv("./normal_data/Y_%02d.csv"%i)
    train_x = normal_data.filter(regex='X') # Input : X Featrue
    train_y = normal_data["Y_%02d"%i] # Output : Y Feature
    xgb_model = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
    xgb_model.fit(train_x,train_y)
    print(train_x.shape,test_x.shape)
    # print("Y_%02d"%i,"를 설명하는 중요한 X 변수")
    
    # xgboost.plot_importance(xgb_model)
    #plt.show()
    predictions = xgb_model.predict(test_x)
    predictions = pd.DataFrame(predictions)
    print(predictions)
    Y_pred = pd.concat([Y_pred,predictions],axis = 1)

Y_pred.columns = ['ID','Y_01', 'Y_02', 'Y_03', 'Y_04','Y_05', 'Y_06','Y_07', 'Y_08', 'Y_09', 'Y_10','Y_11', 'Y_12','Y_13', 'Y_14']
Y_pred.to_csv("./normal_data_pred/Y_pred.csv",index=False)


(38131, 56) (39608, 56)
              0
0      1.394289
1      1.477549
2      1.401207
3      1.412312
4      1.366602
...         ...
39603  1.304819
39604  1.277769
39605  1.213499
39606  1.220327
39607  1.260134

[39608 rows x 1 columns]
(39049, 56) (39608, 56)
              0
0      1.204419
1      1.176787
2      1.026753
3      1.139798
4      1.053268
...         ...
39603  0.980183
39604  0.863574
39605  0.994933
39606  0.931647
39607  1.045695

[39608 rows x 1 columns]
(39143, 56) (39608, 56)
              0
0      1.115515
1      1.135581
2      1.136758
3      1.038018
4      0.902410
...         ...
39603  1.012603
39604  0.969751
39605  0.951227
39606  0.984371
39607  0.987382

[39608 rows x 1 columns]
(39107, 56) (39608, 56)
               0
0      14.641161
1      13.654800
2      13.803294
3      15.060390
4      15.338194
...          ...
39603  12.640126
39604  14.446024
39605  13.040938
39606  13.474467
39607  13.061139

[39608 rows x 1 columns]
(39516, 56) (39608, 

### score 확인용

In [31]:
Y_pred = pd.DataFrame()
test_x = pd.read_csv("./test.csv").drop(columns=['ID'])
a = pd.read_csv("./train.csv").drop(columns=['ID'])
a = a.filter(regex="X")
for i in range(1,15):
    normal_data = pd.read_csv("./normal_data/Y_%02d.csv"%i)
    train_x = normal_data.filter(regex='X') # Input : X Featrue
    train_y = normal_data["Y_%02d"%i] # Output : Y Feature
    xgb_model = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
    xgb_model.fit(train_x,train_y)
    print(train_x.shape,test_x.shape)
    plt.show()
    predictions = xgb_model.predict(a)
    predictions = pd.DataFrame(predictions)
    print(predictions)
    Y_pred = pd.concat([Y_pred,predictions],axis = 1)

Y_pred.columns = ['Y_01', 'Y_02', 'Y_03', 'Y_04','Y_05', 'Y_06','Y_07', 'Y_08', 'Y_09', 'Y_10','Y_11', 'Y_12','Y_13', 'Y_14']
Y_pred.to_csv("./normal_data_pred/Y_pred.csv",index=False)

(38131, 56) (39608, 56)
              0
0      1.355409
1      1.471479
2      1.310230
3      1.400182
4      1.299545
...         ...
39602  1.283876
39603  1.353519
39604  1.212335
39605  1.130591
39606  1.235781

[39607 rows x 1 columns]
(39049, 56) (39608, 56)
              0
0      1.288341
1      1.147026
2      0.975100
3      1.075619
4      0.909853
...         ...
39602  0.951113
39603  0.913308
39604  0.941884
39605  0.882389
39606  0.880643

[39607 rows x 1 columns]
(39143, 56) (39608, 56)
              0
0      1.341120
1      1.145489
2      0.958469
3      1.067447
4      0.969637
...         ...
39602  1.102878
39603  1.019586
39604  0.896157
39605  0.934809
39606  0.921801

[39607 rows x 1 columns]
(39107, 56) (39608, 56)
               0
0      12.727301
1      16.177141
2      14.580446
3      15.588706
4      15.167845
...          ...
39602  13.020772
39603  12.859164
39604  13.731475
39605  12.465792
39606  13.109242

[39607 rows x 1 columns]
(39516, 56) (39608, 

In [37]:
a = pd.read_csv("./train.csv").drop(columns=['ID'])
valid_x = a.filter(regex='X') # Input : X Featrue
valid_y = a.filter(regex='Y') # Output : Y Feature
valid_preds = pd.read_csv("./normal_data_pred/Y_pred.csv")

score = lg_nrmse(valid_y.values,valid_preds.values)
score

1.7355972148727803

# 끝